In [ ]:
!pip install transformers 

     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 51.7 MB/s 
     |████████████████████████████████| 895 kB 50.7 MB/s 
     |████████████████████████████████| 61 kB 402 kB/s 
     |████████████████████████████████| 3.3 MB 54.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
import re
import string
import tensorflow as tf 
import pickle
from timeit import default_timer as timer  
from sklearn.model_selection import train_test_split

In [ ]:
train_df = pickle.load(open('/content/drive/MyDrive/Copy of train_df_prep.pkl','rb'))

In [ ]:
x_train, x_val , y_train , y_val = train_test_split(train_df[['text','sentiment']],train_df['selected_text'],test_size=0.2, random_state=42)
x_train.shape, x_val.shape , y_train.shape , y_val.shape

((21976, 2), (5495, 2), (21976,), (5495,))

In [ ]:
stweet = x_val.values[200]       # tweet 
stweet

array(['enzo blech thats a fail when youre receiving dollars i quoted a job in usd last month im getting paid now  major losses',
       'negative'], dtype=object)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',add_prefix_space=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
pickle.dump(tokenizer,open('/content/drive/MyDrive/quantization/tokenizer.pkl','wb'))
tokenizer = pickle.load(open('/content/drive/MyDrive/quantization/tokenizer.pkl','wb')

In [ ]:
def text_cleaning(text):
  '''' takes input as raw  text  and removes hyperlinks,Numbers,Angular Brackets,Square Brackets,'\n' character,**** by ABUSE,wordpuntuation '''
  text = str(text).lower()
  text = re.sub('https?://\S+|www\.\S+', '', text)  #Removing hyperlinks
  text=re.sub('\S*\d\S*',' ',text) #Removing Numbers
  text=re.sub('<.*?>+',' ',text)   #Removing Angular Brackets
  text=re.sub('\[.*?\]',' ',text)  #Removing Square Brackets
  text=re.sub('\n',' ',text)       #Removing '\n' character 
  text=re.sub('\*+','<ABUSE>',text) #Replacing **** by ABUSE word

  text = "".join([i for i in text if i not in string.punctuation]) # Removing puntuation 
  return text

In [ ]:
def prediction_func1(stweet):
  ''' function takes twee-text and tweet-sentiment as input and outputs phrase in the text which supports sentiment '''
  start = timer()
    
  tweet          = text_cleaning(stweet[0])
  encoder        = tokenizer.encode_plus(tweet,stweet[1],add_special_tokens=True,max_length=92,
                              return_attention_mask=True,pad_to_max_length=True,return_tensors='tf',verbose=False)
  input_ids      = encoder['input_ids'] 
  attention_mask = encoder['attention_mask']
  interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/quantization/converted_quant_model.tflite")
  interpreter.allocate_tensors()
  # Get input and output tensors.
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  # Test model on some input data.
  input_shape = input_details[0]['shape']

  interpreter.set_tensor(input_details[1]['index'], input_ids)
  interpreter.set_tensor(input_details[0]['index'], attention_mask)
  interpreter.invoke()
  start_logits = interpreter.get_tensor(output_details[0]['index'])
  end_logits  = interpreter.get_tensor(output_details[1]['index'])

  a = np.argmax(start_logits)
  b = np.argmax(end_logits)
  text1 = " "+" ".join(tweet.split())               # pred_answer
  enc = tokenizer.encode(text1)
  pred_text = tokenizer.decode(enc[a:b+1]).replace('[SEP]','')
  end = timer()
  print('total time : ',end - start)
  return pred_text

pred = prediction_func1(stweet)
pred

total time :  1.2565918199998123


'a fail when'

In [ ]:
pred

'a fail when'

In [ ]:
y_val.values[200]

'fail'

In [ ]:
def jaccard_function2(str1,str2): 
  ''' takes input as two strings and outputs jaccard score'''
  a=set(str1.lower().split()) 
  b=set(str2.lower().split())
  c=a.intersection(b)
  return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
jaccard_function2(y_val.values[200],pred)

0.3333333333333333